# Deep Learning with TensorFlow Datasets

We work through an end-to-end deep learning experiment with a large and complex TFDS.

# Import **tensorflow** Library

Import library and alias it:

In [ ]:
import tensorflow as tf

# GPU Hardware Accelerator

To vastly speed up processing, we can use the GPU available from the Google Colab cloud service. Colab provides a free Tesla K80 GPU of about 12 GB. It’s very easy to enable the GPU in a Colab notebook:

1.	click **Runtime** in the top left menu
2.	click **Change runtime** type from the drop-down menu
3.	choose **GPU** from the Hardware accelerator drop-down menu
4.	click **SAVE**

# Test if GPU is Active

Verify that the GPU is active in this notebook:

In [ ]:
tf.__version__, tf.test.gpu_device_name()

Import the **tensorflow** library. If '/device:GPU:0' is displayed, the GPU is active. If '..' is displayed, the regular CPU is active.

# Train **cats_vs_dogs** TFDS

Load **cats_vs_dogs** dataset with simple parameters to get its metadata:

In [ ]:
import tensorflow_datasets as tfds

data, info = tfds.load(name='cats_vs_dogs', with_info=True,
                       try_gcs=True)

The dataset contains a large set of images of cats and dogs.

## Access Metadata

Display contents of the **info** object:

In [ ]:
info

Get labels and number of classes:

In [ ]:
class_labels = info.features['label'].names
num_classes = info.features['label'].num_classes
class_labels, num_classes

Get number of train, validation, and test examples for a 80/10/10 split:

In [ ]:
num_train_img = info.splits['train[0%:80%]'].num_examples
num_validation_img = info.splits['train[80%:90%]'].num_examples
num_test_img = info.splits['train[90%:100%]'].num_examples
print ('train images:', num_train_img)
print ('validation images:', num_validation_img)
print ('test images:', num_test_img)

Validate number of examples in each split:

In [ ]:
train_num = num_train_img /23262
validation_num = num_validation_img /23262
test_num = num_test_img /23262

'{0:.0%}'.format(train_num), '{0:.0%}'.format(validation_num),\
'{0:.0%}'.format(test_num)

## Split Data

Now that we know the splits for the dataset, we can split the dataset or load it with the splits that we want. Let's just load the dataset with an 80/10/10 split:

In [ ]:
(training_set, validation_set, test_set), info = tfds.load(
    'cats_vs_dogs', with_info=True,
    split=['train[:80%]', 'train[80%:90%]',
           'train[90%:]'], shuffle_files=True,
    as_supervised=True, try_gcs=True)

The split is 80% train and 10% each for validation and test data.

Just to be sure, manually check splits:

In [ ]:
len(list(training_set)), len(list(validation_set)),\
len(list(test_set))

## Display Examples

Display some examples with **show_examples**:

In [ ]:
fig = tfds.show_examples(training_set, info)

Display as a dataframe:

In [ ]:
tfds.as_dataframe(training_set.take(4), info)

Display manually. Begin by taking some examples:

In [ ]:
images, labels = [], []
for img, lbl in training_set.take(4):
  img = tf.squeeze(img)
  images.append(img), labels.append(lbl)

Visualize:

In [ ]:
import matplotlib.pyplot as plt

rows, cols = 2, 2
plt.figure(figsize=(10, 10))
for i in range(rows*cols):
  plt.subplot(rows, cols, i + 1)
  plt.imshow(images[i], cmap='bone')
  t = class_labels[labels[i]] + ' (' +\
      str(labels[i].numpy()) + ')'
  plt.title(t)
  plt.axis('off')

## Display Element Specification for Train Set

Display train set information with **element_spec**:

In [ ]:
training_set.element_spec

## Inspect Examples

Take some examples and convert them to numpy:

In [ ]:
features, labels = [], []
for img, lbl in training_set.take(4):
  img = tfds.as_numpy(img)
  lbl = tfds.as_numpy(lbl)
  features.append(img)
  labels.append(lbl)

Create empty lists. Take four examples, convert TFDS images and labels to numpy with **tfds.as_numpy**, and append to lists.

Display examples:

In [ ]:
rows, cols = 2, 2
plt.figure(figsize=(10, 10))
for i in range(rows*cols):
  c = class_labels[labels[i]]
  s = str(features[i].shape)
  title = c + ' ' + s
  plt.subplot(rows, cols, i + 1)
  plt.title(title)
  plt.imshow(features[i], cmap='binary')
  plt.axis('off')

Images have different shapes. So we must resize them to the same shape.

## Reformat Images

We can resize images to any size, but it is faster to train a model with smaller images. So resize images to 150 × 150 pixels. The function resizes and scales images.

In [ ]:
def format_image(image, label):
  image = tf.image.resize(image, (150, 150))/255.0
  return image, label

## Configure Dataset for Performance

Use buffered prefetching and caching to improve I/O performance.

Prefetching overlaps the preprocessing and model execution of a training step. While the model is executing training step **s**, the input pipeline is reading the data for step **s+1**. Doing so reduces the step time to the maximum (as opposed to the sum) of the training and the time it takes to extract the data. The tf.Dataset.prefetch transformation overlaps data preprocessing and model execution while training.

The tf.data.Dataset.cache transformation can cache a dataset, either in memory or on local storage. This save some operations (like file opening and data reading) from being executed during each epoch. Specifically, Dataset.cache keeps the images in memory after they're loaded off disk during the first epoch. This ensures that the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache.

Resource:

https://www.tensorflow.org/guide/data_performance

## Prepare Data for TensorFlow Consumption

Shuffle train data. Resize and scale train, validation, and test data by mapping **format_image** onto the datasets. Batch, cache, and prefetch the datasets.

In [ ]:
BATCH_SIZE = 200
SHUFFLE_SIZE = 500

train_batches = training_set.shuffle(SHUFFLE_SIZE).\
map(format_image).batch(BATCH_SIZE).cache().prefetch(1)

validation_batches = validation_set.\
map(format_image).batch(BATCH_SIZE).cache().prefetch(1)

test_batches = test_set.\
map(format_image).batch(BATCH_SIZE).cache().prefetch(1)

Display element specification for training batches:

In [ ]:
train_batches.element_spec

Inspect all tensors:

In [ ]:
train_batches, validation_batches, test_batches

## Visualize Images from a Batch

Grab the first batch from the training set:

In [ ]:
for img, lbl in train_batches.take(1):
  print (img.shape)

Inspect the first image from the batch:

In [ ]:
img[0].shape, class_labels[lbl[0].numpy()]

Grab some images from the batch:

In [ ]:
images, labels = [], []
for i in range(4):
  tf.squeeze(img[i])
  images.append(img[i]), labels.append(lbl[i])

Visualize:

In [ ]:
rows, cols = 2, 2
plt.figure(figsize=(10, 10))
for i in range(rows*cols):
  plt.subplot(rows, cols, i + 1)
  plt.imshow(images[i], cmap='bone')
  t = class_labels[labels[i]] + ' (' +\
      str(labels[i].numpy()) + ')'
  plt.title(t)
  plt.axis('off')


## Create the Model

Place input shape into a variable:

In [ ]:
for img, lbl in train_batches.take(1):
  in_shape = img.shape[1:]
in_shape

Import libraries:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,\
Dense, Flatten, Dropout

Clear previous models and generate seed:

In [ ]:
import numpy as np

tf.keras.backend.clear_session()
np.random.seed(0)
tf.random.set_seed(0)

Build the Model:

In [ ]:
model = Sequential([
  Conv2D(32, (3, 3), activation = 'relu',
         input_shape=in_shape, strides=1,
         kernel_regularizer='l1_l2'),
  MaxPooling2D(2, 2),
  Conv2D(64, (3, 3), activation='relu'),
  MaxPooling2D(2, 2),
  Conv2D(128, (3, 3), activation='relu'),
  MaxPooling2D(2),
  Conv2D(128, (3, 3), activation='relu'),
  MaxPooling2D(2, 2),
  Flatten(),
  Dense(512, activation='relu'),
  Dense(num_classes, activation='sigmoid')
])

Inspect the model with **summary()**:

In [ ]:
model.summary()

## Compile

Compile with **tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)**:

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True)

model.compile(optimizer='adam',
              loss=loss,
              metrics=['accuracy'])

## Train

Train for ten epochs:

In [ ]:
epochs = 10
history = model.fit(train_batches, epochs=epochs,
                    verbose=1,
                    validation_data=validation_batches)

## Generalize

In [ ]:
loss, acc = model.evaluate(test_batches)
print ('loss:', loss)
print ('accuracy:', acc)

## Visualize

Create a function to visualize model performance:

In [ ]:
def viz(hd):
  acc = hd['accuracy']
  val_acc = hd['val_accuracy']
  loss = hd['loss']
  val_loss = hd['val_loss']
  plt.figure(figsize=(8, 8))
  plt.subplot(1, 2, 1)
  plt.plot(acc, label='Training Accuracy')
  plt.plot(val_acc, label='Validation Accuracy')  
  plt.legend(loc='lower right')
  plt.title('Training and Validation Accuracy')
  plt.subplot(1, 2, 2)
  plt.plot(loss, label='Training Loss')
  plt.plot(val_loss, label='Validation Loss')
  plt.legend(loc='upper right')
  plt.title('Training and Validation Loss')
  plt.show()

Invoke:

In [ ]:
viz(history.history)

# Data Augmentation with Keras Preprocessing Layers

To mitigate overfitting and hopefully improve model performance, we use data augmentation. Begin by implementing data augmentation using experimental Keras Preprocessing Layers. Let's try random horizontal flips, random rotation, and random zoom:

In [ ]:
from tensorflow.keras import layers

data_augmentation = tf.keras.Sequential(
  [
    layers.experimental.preprocessing.RandomFlip('horizontal'),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

## Display an Augmented Image

Here is what happens when applying data augmentation to the same image several times:

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_batches.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0])
    plt.axis('off')

## Clear Previous Models and Generate Seed

Clear previous models and generate seed with various tools:

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(0)
tf.random.set_seed(0)

## Build Model

Create a multilayered CNN:

In [ ]:
model = Sequential([
  data_augmentation,
  Conv2D(32, (3, 3), activation = 'relu',
         input_shape=in_shape, strides=1,
         kernel_regularizer='l1_l2'),
  MaxPooling2D(2, 2),
  Conv2D(64, (3, 3), activation='relu'),
  MaxPooling2D(2, 2),
  Conv2D(128, (3, 3), activation='relu'),
  MaxPooling2D(2),
  Conv2D(128, (3, 3), activation='relu'),
  MaxPooling2D(2, 2),
  Flatten(),
  Dense(512, activation='relu'),
  Dense(num_classes, activation='sigmoid')
])

## Compile

Compile with **tf.keras.losses.SparseCategoricalCrossentropy(              from_logits=True)**:

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])

## Train

Train model for ten epochs:

In [ ]:
#tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

epochs = 10
history = model.fit(train_batches, epochs=epochs,
                    verbose=1,
                    validation_data=validation_batches)

## Generalize

In [ ]:
loss, acc = model.evaluate(test_batches)
print ('loss:', loss)
print ('accuracy:', acc)

## Visualize

Visualize model performance:

In [ ]:
viz(history.history)

Validation accuracy has a better trajectory with data augmentation.

# Implement Data Augmentation on Images

Implement data augmentation by performing transformations on images.

Create functions to augment images:

In [ ]:
def random_crop(image):
    shape = tf.shape(image)
    min_dim = tf.reduce_min([shape[0], shape[1]]) * 90 // 100
    return tf.image.random_crop(image, [min_dim, min_dim, 3])

def preprocess(image, label):
  cropped_image = random_crop(image)
  cropped_image = tf.image.random_flip_left_right(cropped_image)
  resized_image = tf.image.resize(cropped_image, [150, 150])
  final_image = tf.keras.applications.xception.preprocess_input(
      resized_image)
  return final_image, label

The random_crop function randomly crops images. The preprocess function crops based on the randomized parameter and resizes image. The **tf.keras.applications.xception.preprocess_input** utility preprocesses a tensor or numpy array by encoding a batch of images.

## Display an Augmented Image

Show what happens to an image with augmentation:

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_batches.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    Images = np.clip(augmented_images, 0, 1)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(Images[0])
    plt.axis('off')

## Build the Input Pipeline

Import the **partial** package, set batch and shuffle size, and build the pipeline: 

In [ ]:
from functools import partial

BATCH_SIZE = 200
SHUFFLE_SIZE = 500

train_shuffle = training_set.shuffle(1000)
train_batches = train_shuffle.map(partial(preprocess)).\
  batch(BATCH_SIZE).prefetch(1)
validation_batches = validation_set.map(preprocess).\
  batch(BATCH_SIZE).prefetch(1)
test_batches = test_set.map(preprocess).\
  batch(BATCH_SIZE).prefetch(1)

Partial functions fix a certain number of arguments of a function and generate a new function.

## Clear Previous Models and Generate Seed

Clear previous models and seed with various tools:

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(0)
tf.random.set_seed(0)

## Model

Create a multilayered CNN:

In [ ]:
model = Sequential([
  Conv2D(32, (3, 3), activation = 'relu',
         input_shape=in_shape, strides=1,
         kernel_regularizer='l1_l2'),
  MaxPooling2D(2, 2),
  Conv2D(64, (3, 3), activation='relu'),
  MaxPooling2D(2, 2),
  Conv2D(128, (3, 3), activation='relu'),
  MaxPooling2D(2),
  Conv2D(128, (3, 3), activation='relu'),
  MaxPooling2D(2, 2),
  Flatten(),
  Dense(512, activation='relu'),
  Dense(num_classes, activation='sigmoid')
])

## Compile

Compile with **tf.keras.losses.SparseCategoricalCrossentropy(                 from_logits=True)**:

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(
                  from_logits=True),
              metrics=['accuracy'])

## Train

Train:

In [ ]:
epochs = 10
history = model.fit(train_batches, epochs=epochs,
                    verbose=1,
                    validation_data=validation_batches)

## Generalize

In [ ]:
loss, acc = model.evaluate(test_batches)
print ('loss:', loss)
print ('accuracy:', acc)

## Visualize

Visualize model performance:

In [ ]:
viz(history.history)

## Predictions

We can make predictions on the whole dataset:

In [ ]:
predictions = model.predict(test_batches)
list(predictions[0])

Predict on 'test_batches' because it has never been seen by the model. The *predict()* method returns NumPy arrays of predictions. For classification experiments, each element in a prediction array represents a class label. In our experiment, we have two classes (cats and dogs). So each prediction array has two elements. The value of each element is a probability. The magnitude of the probability represents the likelihood that the element is the predicted class.

Return the prediction for the first example: 

In [ ]:
first_prediction = tf.math.argmax(predictions[0])
class_labels[first_prediction.numpy()]

We can also make predictions on a single batch:

In [ ]:
first_batch = test_batches.take(1)
predict_batch = model.predict(first_batch)

Get the first prediction:

In [ ]:
first_batch_prediction = tf.math.argmax(predict_batch[0])
class_labels[first_batch_prediction.numpy()]

Get the first actual label:

In [ ]:
for image, label in first_batch:
  print ('batch size:', image.shape[0])
class_labels[label[0].numpy()]

If the first prediction matches the first actual label, the prediction is correct!

Manually check the prediction accuracy of the first batch:

In [ ]:
cnt = 0
for i in range(image.shape[0]):
  pred = tf.math.argmax(predict_batch[i]).numpy()
  actual = label[i].numpy()
  if actual == pred:
    cnt += 1
acc = cnt / image.shape[0]
'{percent:.2%}'.format(percent=acc) + ' accuracy'

## Visualize Predictions

Grab the first batch of images and labels:

In [ ]:
for img, lbl in test_batches.take(1):
  print (img.shape)

Each batch contains 200 150 x 150 color images.

Inspect the first image from the first batch:

In [ ]:
img[0].shape, class_labels[lbl[0].numpy()]

Visualize the image:

In [ ]:
Image= np.clip(img[0], 0, 1)
fig = plt.imshow(Image)
fig = plt.axis('off')

Process some examples:

In [ ]:
images, labels = [], []
for i in range(20):
  tf.squeeze(img[i])
  images.append(img[i]), labels.append(lbl[i])

We have 20 imaes and labels:

In [ ]:
len(images), len(labels)

Create a function to display a set of images and labels. The function determines if a prediction is correct or incorrect. Correct predictions are displayed normally, but incorrect predictions are displayed in red text.

In [ ]:
def display_test(feature, target, num_images,
                 n_rows, n_cols, cl, p):
  for i in range(num_images):
    plt.subplot(n_rows, 2*n_cols, 2*i+1)
    Image= np.clip(feature[i], 0, 1)
    plt.imshow(Image)
    pred = cl[tf.math.argmax(p[i]).numpy()]
    actual = cl[target[i]]
    title_obj = plt.title(actual + ' (' +\
                          pred + ') ')
    if pred == actual:
      title_obj
    else:
      plt.getp(title_obj, 'text')
      plt.setp(title_obj, color='r')
    plt.tight_layout()
    plt.axis('off')

Invoke the function for the images we processed:

In [ ]:
num_rows, num_cols = 5, 4
num_images = num_rows*num_cols
plt.figure(figsize=(20, 20))
display_test(images, labels, num_images, num_rows,
             num_cols, class_labels, predictions)

# Train **rock_paper_scissors** TFDS

The dataset contains images of hands playing the rock-paper-scissors game.

For an excellent tutorial on training the dataset, peruse:

https://colab.research.google.com/github/trekhleb/machine-learning-experiments/blob/master/experiments/rock_paper_scissors_cnn/rock_paper_scissors_cnn.ipynb

## Configure TensorBoard

**TensorBoard** is a tool that provides measurements and visualizations during the machine learning workflow. It tracks experiment metrics including loss, accuracy, model graph visualization, and embedding projections to a lower dimensional space.

For a nice tutorial, peruse:

https://www.tensorflow.org/tensorboard/get_started

Load the TensorBoard notebook extension:

In [ ]:
%load_ext tensorboard

Import requisite library:

In [ ]:
import datetime

Clear logs from previous runs:

In [ ]:
!rm -rf ./logs/

## Load Data

Load the train and test set:

In [ ]:
(train_digits, test_digits), rps_info = tfds.load(
    'rock_paper_scissors', with_info=True,
    data_dir='tmp', as_supervised=True,
    split=['train', 'test'])

## Inspect Data

Get metadata:

In [ ]:
rps_info

Visualize:

In [ ]:
fig = tfds.show_examples(train_digits, rps_info)

Get number of examples and class labels:

In [ ]:
train_examples = rps_info.splits['train'].num_examples
test_examples = rps_info.splits['test'].num_examples
num_labels = rps_info.features['label'].num_classes
train_examples, test_examples, num_labels

Get shapes:

In [ ]:
rps_info.features['image'].shape,\
rps_info.features['label'].shape 

Get label names:

In [ ]:
label_name = rps_info.features['label'].int2str
for lbl in range(num_labels):
  print (label_name(lbl), end=' ')

Inspect image shape:

In [ ]:
for image, label in train_digits.take(5):
  print (image.shape)

## Preprocess the Data

Reduce image size in half:

In [ ]:
new_pixels = rps_info.features['image'].shape[0] // 2
new_pixels

Create a preprocessing function:

In [ ]:
def format_digits(image, label):
  image = tf.cast(image, tf.float32) / 255.
  image = tf.image.resize(image, [new_pixels, new_pixels])
  return image, label

Although images are of the same size, resize them to use less memory during training. Scale images to be in the \[0, 1] range to improve training performance.

Preprocess train and test sets:

In [ ]:
train_original = train_digits.map(format_digits)
test_original = test_digits.map(format_digits)

Explore an example:

In [ ]:
for image, label in train_original.take(1):
  finger_img_shape = image.shape
  print (image.shape, image[0][0].numpy(), label.numpy())

## Visualize Processed Data

Create a function to visualize data:

In [ ]:
def preview_dataset(dataset):
  plt.figure(figsize = (12, 12))
  plot_index = 0
  for image, label in dataset.take(12):
    plot_index += 1
    plt.subplot(3, 4, plot_index)
    plt.axis('Off')
    label = label_name(label.numpy())
    plt.title(label)
    plt.imshow(image.numpy())

Invoke the function:

In [ ]:
preview_dataset(train_original)

## Augment Training Data

### Create Data Augmentation Functions

Images coming into the functions are transformed to TensorFlow tensors by the **tf.Tensor** API.

Create a function to randomly flip images:

In [ ]:
def flip(image: tf.Tensor) -> tf.Tensor:
  image = tf.image.random_flip_left_right(image)
  image = tf.image.random_flip_up_down(image)
  return image

Create a function to randomly augment color and clip the image to the \[0, 1] range:

In [ ]:
def color(image: tf.Tensor) -> tf.Tensor:
  image = tf.image.random_hue(
      image, max_delta=0.2)
  image = tf.image.random_saturation(
      image, lower=0.7, upper=1.3)
  image = tf.image.random_brightness(image, 0.05)
  image = tf.image.random_contrast(
      image, lower=0.8, upper=1)
  image = tf.clip_by_value(
      image, clip_value_min=0, clip_value_max=1)
  return image

Create a function to randomly rotate an image:

In [ ]:
def rotate(image: tf.Tensor) -> tf.Tensor:
  return tf.image.rot90(
      image,
      tf.random.uniform(
          shape=[], minval=0,
          maxval=4, dtype=tf.int32))

Create a function to randomly invert an image:

In [ ]:
def invert(image: tf.Tensor) -> tf.Tensor:
  random = tf.random.uniform(
      shape=[], minval=0, maxval=1)
  if random > 0.5:
    image = tf.math.multiply(image, -1)
    image = tf.math.add(image, 1)
  return image

Create a function to zoom an image:

In [ ]:
def zoom(
    image: tf.Tensor, min_zoom=0.8, max_zoom=1.0) -> tf.Tensor:
  image_width, image_height, image_colors = image.shape
  crop_size = (image_width, image_height)
  scales = list(np.arange(min_zoom, max_zoom, 0.01))
  boxes = np.zeros((len(scales), 4))
  for i, scale in enumerate(scales):
    x1 = y1 = 0.5 - (0.5 * scale)
    x2 = y2 = 0.5 + (0.5 * scale)
    boxes[i] = [x1, y1, x2, y2]
  def random_crop(img):
    crops = tf.image.crop_and_resize(
        [img], boxes=boxes,
        box_indices=np.zeros(len(scales)),
        crop_size=crop_size)
    return crops[tf.random.uniform(shape=[],
                 minval=0, maxval=len(scales),
                 dtype=tf.int32)]
  choice = tf.random.uniform(shape=[], minval=0., maxval=1., dtype=tf.float32)
  return tf.cond(choice < 0.5, lambda: image, lambda: random_crop(image))

The function generates crop settings ranging from 1% to 20%. It then creates bounding boxes to hold the cropped images. The returned cropped images is resized to keep the size uniform for training. Cropping is only performed 50% of the time.

Create an augment function:

In [ ]:
def augment_data(image, label):
  image = flip(image)
  image = color(image)
  image = rotate(image)
  image = zoom(image)
  image = invert(image)
  return image, label

## Augment Train Data

Map the augmentations to the train data:

In [ ]:
train_augmented = train_original.map(augment_data)

Visualize the augmentated train data:

In [ ]:
preview_dataset(train_augmented)

Visualize the original test data:

In [ ]:
preview_dataset(test_original)

## Build the Input Pipeline

Instantiate train and test sets:

In [ ]:
BATCH_SIZE = 32

train_fingers = train_augmented.shuffle(train_examples).cache().\
  batch(BATCH_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

test_fingers = test_original.batch(BATCH_SIZE)

Prefetch enables the input pipeline to asynchronously fetch batches while the model is training.

## Create the Model

Clear and seed:

In [ ]:
tf.keras.backend.clear_session()
np.random.seed(0)
tf.random.set_seed(0)

Verify image shape:

In [ ]:
finger_img_shape

Build the model:

In [ ]:
finger_model = Sequential([
  Conv2D(64, 3, activation='relu',
         input_shape=finger_img_shape,
         kernel_regularizer='l1_l2'),
  MaxPooling2D(2, 2),
  Conv2D(64, 3, activation='relu'),
  MaxPooling2D(2, 2),
  Conv2D(128, 3, activation='relu'),
  MaxPooling2D(2, 2),
  Conv2D(128, 3, activation='relu'),
  MaxPooling2D(2, 2),
  Flatten(),
  Dense(512, activation='relu'),
  Dense(num_labels, activation='softmax')])

Check the diagram:

In [ ]:
tf.keras.utils.plot_model(
    finger_model,
    show_shapes=True,
    show_layer_names=True)

## Compile and Train

Compile:

In [ ]:
optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.001)

finger_model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=['accuracy'])

Establish training parameters:

In [ ]:
steps_per_epoch = train_examples // BATCH_SIZE
validation_steps = test_examples // BATCH_SIZE

print('steps_per_epoch:', steps_per_epoch)
print('validation_steps:', validation_steps)

Remove logs and checkpoints:

In [ ]:
!rm -rf tmp/checkpoints
!rm -rf logs

Prepare TensorBoard callback:

In [ ]:
log_dir = 'logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1)

Train:

In [ ]:
training_history = finger_model.fit(
    train_fingers.repeat(),
    validation_data=test_fingers.repeat(),
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[tensorboard_callback])

## Visualize Performance

Create a function:

In [ ]:
def viz_history(training_history):
  loss = training_history.history['loss']
  val_loss = training_history.history['val_loss']
  accuracy = training_history.history['accuracy']
  val_accuracy = training_history.history['val_accuracy']
  plt.figure(figsize=(14, 4))
  plt.subplot(1, 2, 1)
  plt.title('Loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.plot(loss, label='Training set')
  plt.plot(val_loss, label='Test set', linestyle='--')
  plt.legend()
  plt.grid(linestyle='--', linewidth=1, alpha=0.5)
  plt.subplot(1, 2, 2)
  plt.title('Accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.plot(accuracy, label='Training set')
  plt.plot(val_accuracy, label='Test set', linestyle='--')
  plt.legend()
  plt.grid(linestyle='--', linewidth=1, alpha=0.5)
  plt.show()

Invoke:

In [ ]:
viz_history(training_history)

Launch TensorBoard:

In [ ]:
%tensorboard --logdir logs/fit

## Close TensorBoard Server

Use the Global Regular Expression Print (grep) command to find the **pid**:

In [ ]:
!ps -ef | grep tensorboard

The **pid** is the first process number:

In [ ]:
!kill 10757